# Credit scoring classification
Ce fichier est un exemple pour le cours de Statistiques pour Big Data du M1 GDA à Nanterre. Il a été fait pendant une heure de cours et est loin d'être efficient aussi bien dans le code que dans la manière de traiter le problème posé. L'idée est de montrer le process à suivre pour une analyse lorsque l'on possède beaucoup de variables explicatives dans un cadre prédictif. 


## 1. Setup

In [223]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix 

## 2. Import data 

In [213]:
df = pd.read_csv('loan_data_2007_2014.csv')

/home/user1/anaconda3/envs/sentiment/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## 3. Exploration

In [214]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466285 entries, 0 to 466284
Data columns (total 74 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           466285 non-null  int64  
 1   member_id                    466285 non-null  int64  
 2   loan_amnt                    466285 non-null  int64  
 3   funded_amnt                  466285 non-null  int64  
 4   funded_amnt_inv              466285 non-null  float64
 5   term                         466285 non-null  object 
 6   int_rate                     466285 non-null  float64
 7   installment                  466285 non-null  float64
 8   grade                        466285 non-null  object 
 9   sub_grade                    466285 non-null  object 
 10  emp_title                    438697 non-null  object 
 11  emp_length                   445277 non-null  object 
 12  home_ownership               466285 non-null  object 
 13 

### 1. Colonnes à supprimer

In [215]:
f_to_delete = ["id", "member_id", "zip_code", "url", "title", "desc", "emp_title", 
               "addr_state", "next_pymnt_d", "collection_recovery_fee", "application_type"]
df.drop(columns=f_to_delete, inplace=True)


### 2. Missing values

In [216]:
# Missing values 
df.dropna(thresh = df.shape[0]*0.2, how = 'all', inplace = True, axis=1 )

### 3. Variables expliquée (target)  

In [217]:
df['loan_status'].value_counts(normalize=True)
df['default'] = np.where(df.loc[:, 'loan_status'].isin(['Default', 'Late (16-30 days)',
                                                       'Late (31-120 days)', 
                                                       'Charged Off',
                                                       'Does not meet the credit policy. Status:Charged Off']),-1,1)
df['default']

0         1
1        -1
2         1
3         1
4         1
         ..
466280    1
466281   -1
466282    1
466283    1
466284    1
Name: default, Length: 466285, dtype: int64

### 4. Train Test Split

In [218]:
y = df['default']
X = df.drop('default', axis=1)
X = X.drop('loan_status', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

### 5. Date

In [219]:
def date_col(df, column): 
    df[column] = pd.to_datetime(df[column], format = '%b-%y')
    # Creation variable qui dit depuis combien de temps le client n'a pas payé 
    date_ref = pd.to_datetime('2018-12-01')
    df['depuis_'+ column] = pd.to_numeric((date_ref-df[column])/ np.timedelta64(1, 'M'))
    df.drop(columns = [column], inplace = True)
    
date_col(X_train, 'last_pymnt_d')
date_col(X_train, 'last_credit_pull_d')
date_col(X_train, 'issue_d')
date_col(X_train, 'earliest_cr_line')
X_train.info()

X_train.drop('term', axis=1, inplace=True)

/home/user1/anaconda3/envs/sentiment/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/user1/anaconda3/envs/sentiment/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/user1/anaconda3/envs/sentiment/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://panda

<class 'pandas.core.frame.DataFrame'>
Int64Index: 349713 entries, 76820 to 201841
Data columns (total 44 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   loan_amnt                    349713 non-null  int64  
 1   funded_amnt                  349713 non-null  int64  
 2   funded_amnt_inv              349713 non-null  float64
 3   term                         349713 non-null  object 
 4   int_rate                     349713 non-null  float64
 5   installment                  349713 non-null  float64
 6   grade                        349713 non-null  object 
 7   sub_grade                    349713 non-null  object 
 8   emp_length                   333961 non-null  object 
 9   home_ownership               349713 non-null  object 
 10  annual_inc                   349709 non-null  float64
 11  verification_status          349713 non-null  object 
 12  pymnt_plan                   349713 non-null  object 


### 6. Dummies 

In [220]:
X_train_cat = X_train.select_dtypes(include = 'object').copy()

def dummies(df, columns):
    df_d = []
    for col in columns:
        df_d.append(pd.get_dummies(df[col], prefix = col))
    df_d = pd.concat(df_d, axis=1)
    df = pd.concat([df, df_d], axis=1)
    df.drop(columns, axis=1, inplace=True)
    return df

X_train = dummies(X_train, ['grade', 'sub_grade', 'emp_length', 'home_ownership',
                            'verification_status', 'pymnt_plan', 'purpose', 
                            'initial_list_status'])


In [228]:
reg = LogisticRegression(penalty='l1',fit_intercept=True,  
                          class_weight='balanced', solver='saga', l1_ratio=0.9)
X_train.fillna(value=0, axis = 1, inplace=True)
reg.fit(X_train, y_train)

/home/user1/anaconda3/envs/sentiment/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1319: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  "(penalty={})".format(self.penalty))
/home/user1/anaconda3/envs/sentiment/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(class_weight='balanced', l1_ratio=0.9, penalty='l1',
                   solver='saga')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 349713 entries, 249233 to 197014
Data columns (total 43 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   loan_amnt                    349713 non-null  int64  
 1   funded_amnt                  349713 non-null  int64  
 2   funded_amnt_inv              349713 non-null  float64
 3   int_rate                     349713 non-null  float64
 4   installment                  349713 non-null  float64
 5   grade                        349713 non-null  object 
 6   sub_grade                    349713 non-null  object 
 7   emp_length                   334004 non-null  object 
 8   home_ownership               349713 non-null  object 
 9   annual_inc                   349710 non-null  float64
 10  verification_status          349713 non-null  object 
 11  pymnt_plan                   349713 non-null  object 
 12  purpose                      349713 non-null  object 

/home/user1/anaconda3/envs/sentiment/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [230]:
confusion_matrix(y_train, reg.predict(X_train))

array([[ 30909,   8163],
       [  1060, 309581]])

In [231]:
table = pd.DataFrame(columns=['Name'], data = X_train.columns.values)
table['coeff'] = np.transpose(reg.coef_)
print(table)

                       Name         coeff
0                 loan_amnt -1.084440e-04
1               funded_amnt -1.088647e-04
2           funded_amnt_inv -1.045357e-04
3                  int_rate -2.663149e-07
4               installment -4.275392e-06
..                      ...           ...
110  purpose_small_business -1.177540e-09
111        purpose_vacation -1.290269e-10
112         purpose_wedding -7.191110e-11
113   initial_list_status_f -1.048835e-08
114   initial_list_status_w  5.179170e-09

[115 rows x 2 columns]
